In [3]:
#!pip install scikit-image
# !pip install matplotlib


In [1]:
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.ndimage import rotate,interpolation as inter

In [2]:
import cv2
import numpy as np
from PIL import Image

def convert_white_text_to_black(image):
    # Convert to numpy array
    image = np.array(image)
    
    # Define the threshold for "white" (you can adjust 240 depending on your needs)
    threshold = 240
    
    # For grayscale images
    if len(image.shape) == 2:  # Grayscale image
        # Set pixels that are nearly white (greater than threshold) to black (0)
        image[image >= threshold] = 0
    else:  # Color image (RGB)
        # Convert white pixels (where R, G, and B are all close to 255) to black
        white_mask = np.all(image >= threshold, axis=-1)
        image[white_mask] = [0, 0, 0]  # Set white pixels to black
    
    return Image.fromarray(image)

# Example usage:
# Assuming `preprocessed_image` is the image you've already preprocessed (e.g., denoised):
# preprocessed_image = convert_white_text_to_black(preprocessed_image)


In [3]:
#skew correctness
def skew_correctness(img):
    
    # convert to binary
    wd, ht = img.size
    pix = np.array(img.convert('1').getdata(), np.uint8)
    bin_img = 1 - (pix.reshape((ht, wd)) / 255.0)
    plt.imshow(bin_img, cmap='gray')
    plt.savefig('binary.png')
    def find_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        hist = np.sum(data, axis=1)
        score = np.sum((hist[1:] - hist[:-1]) ** 2)
        return hist, score
    delta = 1
    limit = 5
    angles = np.arange(-limit, limit+delta, delta)
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)
    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    
    # correct skew
    data = inter.rotate(bin_img, best_angle, reshape=False, order=0)
    img = Image.fromarray((255 * data).astype("uint8"))
    img.save('skew_corrected.png')
    return img

In [4]:
def binarization(image):
    open_cv_image = np.array(image)
    ret, imgf = cv2.threshold(open_cv_image, 0, 255,cv2.THRESH_BINARY,cv2.THRESH_OTSU)
    return Image.fromarray(imgf)

In [5]:
def binarize_image(image, threshold=128):
    """
    Convert an image to binary (black and white) using a threshold.

    Parameters:
    - image: PIL Image object
    - threshold: Pixel value threshold for binarization (default is 128)

    Returns:
    - Binarized PIL Image object
    """
    return image.point(lambda x: 0 if x < threshold else 255, '1')  # Convert pixels below the threshold to black


In [6]:
def denoised_image(image):
    image=np.array(image)
    
    return Image.fromarray(cv2.fastNlMeansDenoising(image, None, 30, 7, 21))
    
    # return Image.fromarray(cv2.fastNlMeansDenoising(image, h=30))

In [7]:
def adaptive_thresholding(image):
    """
    Apply adaptive thresholding to the image.

    Parameters:
    - image: PIL Image object

    Returns:
    - PIL Image object after adaptive thresholding
    """
    # Convert the PIL Image to OpenCV format
    open_cv_image = np.array(image)
    
    # Apply adaptive thresholding using OpenCV
    thresholded = cv2.adaptiveThreshold(
        open_cv_image,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11,  # Block size
        2    # Constant to subtract from mean
    )
    
    # Convert back to PIL Image format
    return Image.fromarray(thresholded)


In [8]:
def resize_image(image, scale_factor=3):
    """
    Resize the image by a given scale factor.

    Parameters:
    - image: PIL Image object
    - scale_factor: Factor by which to scale the image (default is 3x)

    Returns:
    - Resized PIL Image object
    """
    new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
    resized_image = image.resize(new_size, Image.LANCZOS)  # Use LANCZOS for high-quality resizing
    return resized_image


In [9]:
def equalize_image(image):
    return ImageOps.equalize(image) 

def binarize_image(image, threshold=32):
    """
    Convert an image to binary (black and white) using a threshold.

    Parameters:
    - image: PIL Image object
    - threshold: Pixel value threshold for binarization (default is 128)

    Returns:
    - Binarized PIL Image object
    """
    return image.point(lambda x: 0 if x < threshold else 255, '1')  # Convert pixels below the threshold to black


In [10]:


def sharpen_image(image):
    """
    Sharpen the image to enhance text edges.

    Parameters:
    - image: PIL Image object

    Returns:
    - Sharpened PIL Image object
    """
    return image.filter(ImageFilter.SHARPEN)

def adjust_brightness_contrast(image, brightness_factor=2, contrast_factor=3):
    """
    Adjust brightness and contrast of the image.

    Parameters:
    - image: PIL Image object
    - brightness_factor: Factor to adjust brightness (default is 1.5)
    - contrast_factor: Factor to adjust contrast (default is 2)

    Returns:
    - Adjusted PIL Image object
    """
    # Adjust brightness
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)
    
    # Adjust contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)
    
    return image


In [11]:

def sharpen_image1(image):
    # Create a sharpening kernel
    image=np.array(image)
    kernel = np.array([[0, -1, 0], 
                       [-1, 5,-1], 
                       [0, -1, 0]])
    
    # Apply the kernel to the image
    sharpened = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    return Image.fromarray(sharpened)


In [12]:
def normalize(image):
    """
    Normalize the pixel values of an image to the range 0-255.

    Parameters:
    - image: PIL Image object

    Returns:
    - Normalized PIL Image object
    """
    # Convert PIL Image to numpy array
    image_np = np.array(image).astype(np.float32)

    # Normalize to 0-1 range
    normalized = (image_np - np.min(image_np)) / (np.max(image_np) - np.min(image_np))

    # Scale back to 0-255 and convert to uint8
    normalized = (normalized * 255).astype(np.uint8)

    # Convert back to PIL Image
    return Image.fromarray(normalized)


In [16]:
# from skimage.filters import unsharp_mask

# def sharpen_image(image):
#     """
#     Sharpen the image using unsharp mask from skimage.

#     Parameters:
#     - image: PIL Image object

#     Returns:
#     - Sharpened PIL Image object
#     """
#     # Convert image to numpy array and normalize to 0-1 range
#     image_np = np.array(image).astype(np.float32) / 255.0
    
#     # Apply unsharp mask
#     sharp_image_np = unsharp_mask(image_np, radius=1.0, amount=0.5)
    
#     # Convert back to 0-255 range and uint8 format
#     sharp_image_np = np.clip(sharp_image_np * 255.0, 0, 255).astype(np.uint8)
    
#     # Convert back to PIL Image
#     return Image.fromarray(sharp_image_np)


In [17]:
# #Blind Deconvolution
# import torch
# import torch.nn.functional as F
# from torch.optim import Adam
# import numpy as np

# def blind_deconvolution(blurred_image, num_iterations=1000, kernel_size=5):
#     # Convert image to torch tensor
#     blurred_image=np.array(blurred_image)
#     blurred_tensor = torch.tensor(blurred_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
#     # Initialize kernel and latent image
#     kernel = torch.ones((1, 1, kernel_size, kernel_size), requires_grad=True)
#     latent_image = torch.randn_like(blurred_tensor, requires_grad=True)

#     # Define optimizer
#     optimizer = Adam([latent_image, kernel], lr=0.01)

#     for i in range(num_iterations):
#         optimizer.zero_grad()
        
#         # Convolve the latent image with the kernel
#         blurred_estimate = F.conv2d(latent_image, kernel, padding=kernel_size//2)
        
#         # Compute the loss between observed blurred image and estimate
#         loss = F.mse_loss(blurred_estimate, blurred_tensor)
#         loss.backward()
#         optimizer.step()

#         # Normalize kernel to ensure it's a valid PSF
#         with torch.no_grad():
#             kernel.clamp_(min=0)
#             kernel /= kernel.sum()

#         if i % 100 == 0:
#             print(f"Iteration {i}, Loss: {loss.item()}")

#     # Return the final estimated latent image and kernel
#     return latent_image.detach().squeeze().numpy(), kernel.detach().squeeze().numpy()




In [13]:
def denoising(image):
    image=np.array(image)
    # denoising of image saving it into dst image 
    dst = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15) 
    # Plotting of source and destination image 
    return Image.fromarray(dst)
    

In [14]:
def thinning(image):
    image=np.array(image)
    kernel = np.ones((2,2),np.uint8)
    erosion = cv2.erode(image,kernel,iterations = 1)
    return Image.fromarray(erosion)

In [15]:

def preprocess_image_for_ocr(image_path, scale_factor=5, threshold=10, brightness_factor=1.5, contrast_factor=2):
    """
    Preprocess the image for optimal OCR performance.

    Parameters:
    - image_path: Path to the input image file
    - scale_factor: Factor by which to scale the image (default is 3x)
    - threshold: Pixel value threshold for binarization (default is 50)
    - brightness_factor: Factor to adjust brightness (default is 1.5)
    - contrast_factor: Factor to adjust contrast (default is 2)

    Returns:
    - Preprocessed PIL Image object
    """
    # Open the image
    image = Image.open(image_path)
    # image=denoising(image)
    image = image.convert('L')
    
    
    image = sharpen_image(image)    

    # Resize the image
    image = resize_image(image, scale_factor)   
    
    # Convert to grayscale
    image=denoised_image(image)

    #blind dev
    # image, estimated_kernel = blind_deconvolution(image)
    # image = Image.fromarray(np.uint8(image * 255)) 
    #Equlizes the image
    image=equalize_image(image)

    # Normlize the image    
    image=normalize(image)
    
    
#    #Sharpen the image
   
   
    
#     # Adjust brightness and contrast
    image = adjust_brightness_contrast(image, brightness_factor, contrast_factor)
    
    # Sharpen the image
    
    #Thinning
    
    
    # Apply binary thresholding
    
    # image=adaptive_thresholding(image)
    # image=binarization(image)
    image=thinning(image)
    image=convert_white_text_to_black(image)
    image = binarize_image(image, threshold)    
    # image=skew_correctness(image)
    return image

In [16]:
image_path = '../images/41ADVPQgZOL.jpg'
image_path1 = '../images/51KykmLgc0L.jpg'
image_path2 = '../images/417SThj+SrL.jpg'
image_path3 = '../images/41-NCxNuBxL.jpg'
image_path4 = '../images/81aZ2ozp1GL.jpg'
preprocessed_image = preprocess_image_for_ocr(image_path3)
preprocessed_image.show()  # Show the preprocessed image
preprocessed_image.save("pre.png")

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/nova/snap/code/common/.cache/gio-modules/libgiolibproxy.so
Gtk-Message: 10:46:46.028: Failed to load module "appmenu-gtk-module"
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [22]:
# import cv2
# import numpy as np
# from skimage.restoration import denoise_tv_chambolle
# from matplotlib import pyplot as plt

# # Function to apply unsharp mask (enhances the edges and details)
# def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.5, threshold=0):
#     blurred = cv2.GaussianBlur(image, kernel_size, sigma)
#     sharpened = float(amount + 1) * image - float(amount) * blurred
#     sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
#     sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
#     sharpened = sharpened.round().astype(np.uint8)
#     return sharpened

# # Function to preprocess a blurry image
# def preprocess_blurry_image(image_path):
#     # Load the image
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
#     # Step 1: Apply unsharp mask for edge enhancement
#     sharpened_img = unsharp_mask(img)
    
#     # Step 2: Apply Total Variation (TV) denoising for deblurring
#     tv_deblurred_img = denoise_tv_chambolle(sharpened_img, weight=0.1)
    
#     # Rescale to 0-255 after TV deblurring
#     tv_deblurred_img = (tv_deblurred_img * 255).astype(np.uint8)
    
#     # Step 3: Apply CLAHE for contrast enhancement
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#     enhanced_img = clahe.apply(tv_deblurred_img)

#     # Step 4: Apply adaptive thresholding for binarization
#     # threshold_img = cv2.adaptiveThreshold(
#     #     enhanced_img, 255, 
#     #     cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#     #     cv2.THRESH_BINARY, 
#     #     11, 2
#     # )
#     _,threshold_img = cv2.threshold(enhanced_img, 0, 255,cv2.THRESH_BINARY,cv2.THRESH_OTSU)

#     # Step 5: Optional dilation and erosion to improve character shapes
#     kernel = np.ones((2, 2), np.uint8)
#     processed_img = cv2.dilate(threshold_img, kernel, iterations=1)
#     processed_img = cv2.erode(processed_img, kernel, iterations=1)

#     # Step 6: Save and display the processed image
#     processed_image_path = 'processed_image.png'
#     cv2.imwrite(processed_image_path, processed_img)
    
#     # Display for verification
#     plt.imshow(processed_img, cmap='gray')
#     plt.title('Preprocessed Image for OCR')
#     plt.show()

#     return processed_image_path

# # Example usage
# preprocessed_image = preprocess_blurry_image(image_path)


In [31]:
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# # Function to upscale the image using super resolution
# def super_resolution(image_path):
#     try:
#         # Load the pre-trained ESPCN model for super resolution
#         sr = cv2.dnn_superres.DnnSuperResImpl_create()
#         model_path = 'ESPCN_x4.pb'  # Path to ESPCN model
#         sr.readModel(model_path)
#         sr.setModel('espcn', 4)  # Set the model and scale factor

#         # Read the image
#         img = cv2.imread(image_path)
#         if img is None:
#             raise FileNotFoundError(f"Image not found at {image_path}")

#         # Upscale the image
#         upscaled_img = sr.upsample(img)

#         # Convert to grayscale after upscaling
#         gray_upscaled_img = cv2.cvtColor(upscaled_img, cv2.COLOR_BGR2GRAY)

#         # Save and display the upscaled image
#         upscaled_image_path = 'upscaled_image.png'
#         cv2.imwrite(upscaled_image_path, gray_upscaled_img)

#         plt.imshow(gray_upscaled_img, cmap='gray')
#         plt.title('Super Resolution Image')
#         plt.show()

#         return upscaled_image_path
#     except Exception as e:
#         print(f"Error during super resolution: {e}")
#         return None

# # Function to preprocess the super-resolution image
# def preprocess_super_res_image(image_path):
#     try:
#         # Step 1: Apply Super Resolution
#         upscaled_image_path = super_resolution(image_path)
#         if upscaled_image_path is None:
#             return None

#         # Step 2: Load the upscaled image
#         img = cv2.imread(upscaled_image_path, cv2.IMREAD_GRAYSCALE)
#         if img is None:
#             raise FileNotFoundError(f"Upscaled image not found at {upscaled_image_path}")

#         # Step 3: Apply CLAHE for contrast enhancement
#         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#         enhanced_img = clahe.apply(img)

#         # Step 4: Apply Adaptive Thresholding
#         threshold_img = cv2.adaptiveThreshold(
#             enhanced_img, 255,
#             cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#             cv2.THRESH_BINARY, 11, 2
#         )

#         # Step 5: Dilation and Erosion for text emphasis
#         kernel = np.ones((2, 2), np.uint8)
#         processed_img = cv2.dilate(threshold_img, kernel, iterations=1)
#         processed_img = cv2.erode(processed_img, kernel, iterations=1)

#         # Save and display the processed image
#         processed_image_path = 'processed_super_res_image.png'
#         cv2.imwrite(processed_image_path, processed_img)

#         plt.imshow(processed_img, cmap='gray')
#         plt.title('Preprocessed Super Resolution Image for OCR')
#         plt.show()

#         return processed_image_path
#     except Exception as e:
#         print(f"Error during preprocessing: {e}")
#         return None

# # Example usage
# # image_path = 'your_image_path_here.png'  # Define the actual image path
# preprocessed_image = preprocess_super_res_image(image_path)


Error during super resolution: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1138: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "ESPCN_x4.pb" in function 'cv::dnn::ReadProtoFromBinaryFile'



In [24]:
# !pip install opencv-contrib-python


In [25]:
# # !pip install torch torchvision opencv-python pillow

# import cv2
# print(cv2.__version__)


4.10.0


In [26]:
# import numpy as np
# import matplotlib.pyplot as plt
# from skimage import data, restoration
# from scipy.signal import convolve2d

# # Load the example camera image (already in grayscale)
# image = Image.open(image_path)
# image = Image.open(image_path).convert('L')
# image=  np.array(image)

# # Create a synthetic blur kernel (Point Spread Function - PSF)
# psf = np.ones((5, 5)) / 25  # Simple average (motion blur example)
# blurred_image = convolve2d(image, psf, 'same')

# # Add some noise to simulate a real-world scenario
# noisy_blurred_image = blurred_image + 0.1 * blurred_image.std() * np.random.standard_normal(blurred_image.shape)

# # Apply Richardson-Lucy deconvolution
# deconvolved_image = restoration.richardson_lucy(noisy_blurred_image, psf, num_iter=30)

# # Display results
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 3, 1)
# plt.title('Original Image')
# plt.imshow(image, cmap='gray')

# plt.subplot(1, 3, 2)
# plt.title('Blurred + Noisy Image')
# plt.imshow(noisy_blurred_image, cmap='gray')

# plt.subplot(1, 3, 3)
# plt.title('Deconvolved Image (Richardson-Lucy)')
# plt.imshow(deconvolved_image, cmap='gray')

# plt.show()


In [17]:
import os
def process_images_in_folder(input_folder, output_folder):
    """
    Process all images in the input folder and save them to the output folder.

    Parameters:
    - input_folder: Path to the folder containing input images
    - output_folder: Path to the folder where processed images will be saved
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            processed_image = preprocess_image_for_ocr(image_path)
            processed_image_path = os.path.join(output_folder, filename)
            processed_image.save(processed_image_path)
            print(f"Processed {filename} and saved to {processed_image_path}")

# Define the paths for input and output folders
input_folder = '../images'
output_folder = 'processed_images'

# Process all images in the input folder
process_images_in_folder(input_folder, output_folder)

Processed 51jTe522S2L.jpg and saved to processed_images/51jTe522S2L.jpg
Processed 51BEuVR4ZzL.jpg and saved to processed_images/51BEuVR4ZzL.jpg
Processed 71UN1IxKp4L.jpg and saved to processed_images/71UN1IxKp4L.jpg
Processed 41nblnEkJ3L.jpg and saved to processed_images/41nblnEkJ3L.jpg
Processed 417NJrPEk+L.jpg and saved to processed_images/417NJrPEk+L.jpg
Processed pre.png and saved to processed_images/pre.png
Processed 51cPZYLk2YL.jpg and saved to processed_images/51cPZYLk2YL.jpg
Processed 51KykmLgc0L.jpg and saved to processed_images/51KykmLgc0L.jpg
Processed 81qUmRUUTTL.jpg and saved to processed_images/81qUmRUUTTL.jpg
Processed 51+oHGvSvuL.jpg and saved to processed_images/51+oHGvSvuL.jpg
Processed 51vwYpDz2tL.jpg and saved to processed_images/51vwYpDz2tL.jpg
Processed 51tEop-EBJL.jpg and saved to processed_images/51tEop-EBJL.jpg
Processed 41ADVPQgZOL.jpg and saved to processed_images/41ADVPQgZOL.jpg
Processed 61C+fwVD6dL.jpg and saved to processed_images/61C+fwVD6dL.jpg
Processe

In [28]:
# norm_img = np.zeros((img.shape[0], img.shape[1]))
# img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)

In [29]:
# 

In [30]:
# !pip install --upgrade tensorflow keras
